Importing the libraries

Installing the== cleaning text package for NLP developed by me : https://pypi.org/project/cleantxty/

In [1]:
!pip install cleantxty

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, GlobalAveragePooling1D
import cleantxty
import numpy as np

2023-03-13 16:12:32.584579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 16:12:32.706535: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-13 16:12:32.711500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 16:12:32.711517: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Preparing the textual data 

In [3]:
# Prepare Data
text_data = ['This is the first document.', 'This is the second document.', 'And the third one.', 'Is this the first document?']

# Cleaning Data using cleantxty package

for i in text_data:
    i = cleantxty.clean(i)

Creating dictionary of words / tokenizing them

In [4]:
# Create Vocabulary Dictionary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index

print(word_index)

{'the': 1, 'this': 2, 'is': 3, 'document': 4, 'first': 5, 'second': 6, 'and': 7, 'third': 8, 'one': 9}


Converting text to sequences

In [5]:
# Convert Text to Sequence
sequences = tokenizer.texts_to_sequences(text_data)
padded_sequences = pad_sequences(sequences, padding='post')

print(padded_sequences)

[[2 3 1 5 4]
 [2 3 1 6 4]
 [7 1 8 9 0]
 [3 2 1 5 4]]


Creating the Neural Network

In [6]:
# Define Neural Network
model = Sequential()
model.add(Embedding(len(word_index) + 1, 32, name="embedding"))
model.add(GlobalAveragePooling1D())
#model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320       
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________
2023-03-13 16:12:36.068774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-13 16:12:36.068803: W tensor

Training the Neural Network

In [7]:
# Train the Model
model.fit(np.array(padded_sequences), np.array([1, 1, 0, 1]), epochs=50, batch_size=1, verbose=2)

Epoch 1/50
4/4 - 0s - loss: 0.6944 - accuracy: 0.5000 - 353ms/epoch - 88ms/step
Epoch 2/50
4/4 - 0s - loss: 0.6850 - accuracy: 0.7500 - 4ms/epoch - 995us/step
Epoch 3/50
4/4 - 0s - loss: 0.6742 - accuracy: 1.0000 - 4ms/epoch - 936us/step
Epoch 4/50
4/4 - 0s - loss: 0.6637 - accuracy: 1.0000 - 4ms/epoch - 882us/step
Epoch 5/50
4/4 - 0s - loss: 0.6525 - accuracy: 1.0000 - 4ms/epoch - 991us/step
Epoch 6/50
4/4 - 0s - loss: 0.6426 - accuracy: 1.0000 - 3ms/epoch - 872us/step
Epoch 7/50
4/4 - 0s - loss: 0.6327 - accuracy: 1.0000 - 4ms/epoch - 960us/step
Epoch 8/50
4/4 - 0s - loss: 0.6225 - accuracy: 1.0000 - 4ms/epoch - 910us/step
Epoch 9/50
4/4 - 0s - loss: 0.6131 - accuracy: 1.0000 - 5ms/epoch - 1ms/step
Epoch 10/50
4/4 - 0s - loss: 0.6029 - accuracy: 1.0000 - 6ms/epoch - 1ms/step
Epoch 11/50
4/4 - 0s - loss: 0.5920 - accuracy: 1.0000 - 4ms/epoch - 906us/step
Epoch 12/50
4/4 - 0s - loss: 0.5798 - accuracy: 1.0000 - 4ms/epoch - 935us/step
Epoch 13/50
4/4 - 0s - loss: 0.5689 - accuracy: 1.00

Evaluating the model seeing its accuracy

In [8]:
# Evaluate the Model
loss, accuracy = model.evaluate(np.array(padded_sequences), np.array([1, 1, 0, 1]), verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


Predicting on new text data using the trained model

In [9]:
# Use the Model to Predict
new_text_data = ['This is a new document.']
new_sequences = tokenizer.texts_to_sequences(new_text_data)
new_padded_sequences = pad_sequences(new_sequences, padding='post', maxlen=padded_sequences.shape[1])
prediction = model.predict(new_padded_sequences)
print('Prediction: %f' % prediction)

1/1 [==============================] - 0s 53ms/step
Prediction: 0.691805


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=46678de0-d2f0-456a-ad34-9067c644f849' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>